In [6]:
##          DATA SHAPE DEFINITION           ##
import os, json, warnings
warnings.filterwarnings("ignore")

# Define the parameters for the data shape
videos_per_handsign = 150  # Adjust if necessary
frames_per_video = 8
num_landmarks = 65
num_coordinates = 3

num_additional_samples = 1       #how many different samples of frames_per_video frames will be taken from each video (careful to not add too many if vids are short)

num_augmented_versions = 1      #Add noise and transformation to generate, from each video extracted, a num_augmented_versions number of new ones (also keep between 1-4)
if num_augmented_versions>0:
    data_augmentation = True
else: data_augmentation = False

#redefine videos per handsign to accomodate for additional samples to be taken before generating the dummy array (this is done so there's no shaping/dimensions size incompatibility issue)
videos_per_handsign = videos_per_handsign * (1+num_additional_samples)

# Define the root directory containing handsign folders
root_path = "NomenclatedDataset - Copy"

def get_handsign_folders(root_path):
    handsign_names = {}
    handsign_video_counts = {}  # To store the video count for each handsign
    handsign_count = 0
    
    # Walk through the root directory
    for folder in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder)
        
        # Ignore non-directories
        if not os.path.isdir(folder_path):
            continue

        # If the folder starts with "#", check its subdirectories
        if folder.startswith("#"):
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.isdir(subfolder_path):
                    handsign_names[handsign_count] = subfolder
                    # Count videos in the subfolder
                    videos = [f for f in os.listdir(subfolder_path) if f.endswith(('.mp4', '.avi', '.MOV'))]
                    handsign_video_counts[subfolder] = len(videos)
                    handsign_count += 1
        else:
            # Directly add the folder as a handsign
            handsign_names[handsign_count] = folder
            # Count videos in the folder
            videos = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.MOV'))]
            handsign_video_counts[folder] = len(videos)
            handsign_count += 1
    
    return handsign_names, handsign_video_counts

# Get the handsign names and video counts
handsign_names, handsign_video_counts = get_handsign_folders(root_path)
num_handsigns = len(handsign_names)

# Sort handsigns by the number of available videos in descending order
sorted_handsign_video_counts = dict(sorted(handsign_video_counts.items(), key=lambda item: item[1], reverse=True))

# Output the handsign names, number of handsigns, and sorted video counts
print(f"Number of handsigns: {num_handsigns}")
print("Handsign names:")
print(json.dumps(handsign_names, indent=4))

print("\nVideo counts per handsign (sorted by number of videos):")
print(json.dumps(sorted_handsign_video_counts, indent=4))


Number of handsigns: 39
Handsign names:
{
    "0": "A",
    "1": "apellido",
    "2": "aprender",
    "3": "argentina",
    "4": "ayuda",
    "5": "B",
    "6": "C",
    "7": "comprar",
    "8": "cumplea\u00f1os",
    "9": "D",
    "10": "E",
    "11": "F",
    "12": "G",
    "13": "H",
    "14": "hijo",
    "15": "hombre",
    "16": "I",
    "17": "J",
    "18": "K",
    "19": "L",
    "20": "M",
    "21": "mujer",
    "22": "N",
    "23": "nombre",
    "24": "no_sign",
    "25": "O",
    "26": "P",
    "27": "Q",
    "28": "R",
    "29": "S",
    "30": "sordo",
    "31": "T",
    "32": "U",
    "33": "V",
    "34": "W",
    "35": "X",
    "36": "Y",
    "37": "Z",
    "38": "\u00d1"
}

Video counts per handsign (sorted by number of videos):
{
    "G": 278,
    "C": 266,
    "L": 258,
    "Y": 257,
    "I": 251,
    "U": 247,
    "O": 219,
    "J": 205,
    "A": 198,
    "V": 198,
    "H": 182,
    "D": 170,
    "K": 167,
    "W": 165,
    "S": 150,
    "B": 146,
    "cumplea\u00f1os"

In [7]:
import numpy as np

# Generate dummy data
data = [np.random.rand(videos_per_handsign, frames_per_video, num_landmarks, num_coordinates) for _ in range(num_handsigns)]

# Convert the list to a numpy array with shape (num_handsigns, videos_per_handsign, frames_per_video, num_landmarks, num_coordinates)
data_array = np.array(data)

# Save the data array to a .npy file
np.save('handsigns_data.npy', data_array)

In [7]:
##          PROCESS VIDEO DATASET FUNC DEFINITIONS         ##
import cv2
import mediapipe as mp
import numpy as np
import os
from tqdm import tqdm

def extract_landmarks(hands_results, pose_results):
    landmarks = []

    # Extract nose landmark or set to default if not available
    try:
        nose_landmark = pose_results.pose_landmarks.landmark[0]
    except AttributeError:
        class nose_landmark:
            x, y, z = 0, 0, 0
        nose_landmark = nose_landmark()

    # Default placeholders for hand landmarks
    left_hand_landmarks = [(0, 0, 0)] * 21
    right_hand_landmarks = [(0, 0, 0)] * 21

    # Extract hand landmarks if available
    if hands_results.multi_hand_landmarks and hands_results.multi_handedness:
        for i, hand_landmarks in enumerate(hands_results.multi_hand_landmarks):
            label = hands_results.multi_handedness[i].classification[0].label
            # Use the wrist as the reference landmark
            wrist_landmark = hand_landmarks.landmark[0]

            if label == 'Left':
                left_hand_landmarks = [
                    (lm.x - wrist_landmark.x, lm.y - wrist_landmark.y, lm.z - wrist_landmark.z)
                    for lm in hand_landmarks.landmark
                ]
            elif label == 'Right':
                right_hand_landmarks = [
                    (lm.x - wrist_landmark.x, lm.y - wrist_landmark.y, lm.z - wrist_landmark.z)
                    for lm in hand_landmarks.landmark
                ]

    # Add left and right hand landmarks to the landmark list
    landmarks.extend(left_hand_landmarks)
    landmarks.extend(right_hand_landmarks)

    # Add pose landmarks if available, excluding those from the waist down
    if pose_results.pose_landmarks:
        for i, lm in enumerate(pose_results.pose_landmarks.landmark):
            # Exclude landmarks from waist down (indices 23 to 32)
            if i < 23:
                landmarks.append((lm.x - nose_landmark.x, lm.y - nose_landmark.y, lm.z - nose_landmark.z))
    else:
        # Use placeholders for upper body landmarks only
        landmarks.extend([(0, 0, 0)] * 23)

    return np.array(landmarks)




def process_frame(frame, hands, pose):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hands_results = hands.process(image)
    pose_results = pose.process(image)

    landmarks = extract_landmarks(hands_results, pose_results)

    return landmarks


def process_video_with_samples(video_path, frames_per_video, num_landmarks, num_coordinates, num_additional_samples=0):
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not cap.isOpened():
        print(f"Error: Unable to open video {video_path}.")
        return np.zeros((1 + num_additional_samples, frames_per_video, num_landmarks, num_coordinates))
    
    needs_interpolation = False
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if total_frames < frames_per_video/4 or total_frames < 3:
        print(f"Error: frames on {video_path} are less than 25% of frames_per_video or less than 3. filling with zeros.")
        return np.zeros((1 + num_additional_samples, frames_per_video, num_landmarks, num_coordinates))
    elif total_frames < frames_per_video * (1 + num_additional_samples):
        needs_interpolation = True
        print(f"Warning: Video {video_path} has fewer frames ({total_frames}) than required ({frames_per_video} * {num_additional_samples+1}). Interpolating frames.")
        
    # Buffer frames
    frame_buffer = {}
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_buffer[frame_count] = frame
        frame_count += 1
    cap.release()

    # Apply interpolation if needed
    if needs_interpolation:
        interpolated_frames = {}
        for i in range(total_frames - 1):
            interpolated_frames[i] = frame_buffer[i]
            # Create additional interpolated frames between i and i+1
            num_interp_frames = max(1, frames_per_video // total_frames)
            for alpha in np.linspace(0, 1, num_interp_frames + 2)[1:-1]:
                interp_frame = cv2.addWeighted(frame_buffer[i], 1 - alpha, frame_buffer[i + 1], alpha, 0)
                interpolated_frames[len(interpolated_frames)] = interp_frame
        frame_buffer = interpolated_frames

    # Frame selection and sample generation remain the same
    frames_data = []
    frame_sets = [np.linspace(0, len(frame_buffer) - 1, frames_per_video, dtype=int)]

    for i in range(1, num_additional_samples + 1):
        available_frames = list(set(range(len(frame_buffer))) - set(frame_sets[0]))
        if len(available_frames) >= frames_per_video:
            additional_indices = [available_frames[idx] for idx in np.linspace(0, len(available_frames) - 1, frames_per_video, dtype=int)]
        else:
            additional_indices = np.linspace(0, len(frame_buffer) - 1, frames_per_video, dtype=int)
        frame_sets.append(additional_indices)

    # Process the buffered frames for landmarks
    mp_hands = mp.solutions.hands
    mp_pose = mp.solutions.pose

    with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands, \
         mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        
        for frame_set in frame_sets:
            for frame_idx in frame_set:
                if frame_idx in frame_buffer:
                    landmarks = process_frame(frame_buffer[frame_idx], hands, pose)
                    frames_data.append(landmarks)

    # Final reshaping and padding as needed
    reshaped_data = np.array(frames_data).reshape((1 + num_additional_samples, frames_per_video, num_landmarks, num_coordinates))
    
    return reshaped_data

def process_dataset_with_samples(root_path, handsign_names, frames_per_video, num_landmarks, num_coordinates, videos_per_handsign, num_additional_samples=0):
    data = []

    # Calculate how many videos to process based on videos_per_handsign and num_additional_samples (assuming videos_per_handsign was already augmented by num_additional_samples)
    videos_to_process = videos_per_handsign // (1 + num_additional_samples)
    
    for handsign_index in range(len(handsign_names)):
        handsign_folder = handsign_names[handsign_index]
        handsign_path = os.path.join(root_path, handsign_folder)
        
        if not os.path.exists(handsign_path):
            print(f"Warning: Directory {handsign_path} does not exist. Skipping.")
            data.append(np.zeros((videos_per_handsign, frames_per_video, num_landmarks, num_coordinates)))
            continue

        videos = [f for f in os.listdir(handsign_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        handsign_data = []
        
        # Repeat videos if there aren't enough to reach videos_per_handsign
        if len(videos) < videos_to_process:
            print(f"Warning: Handsign '{handsign_folder}' has only {len(videos)} videos. Repeating videos until {videos_to_process} are processed.")
            repeated_videos = (videos * ((videos_to_process // len(videos)) + 1))[:videos_to_process]
        else:
            repeated_videos = videos[:videos_to_process]

        # Process each required video, with repeats if necessary
        for video in tqdm(repeated_videos, desc=f"Processing videos for handsign {handsign_index}", leave=False):
            video_path = os.path.join(handsign_path, video)
            
            # Call the updated process_video_with_samples function
            video_data = process_video_with_samples(video_path, frames_per_video, num_landmarks, num_coordinates, num_additional_samples)

            # Append all generated samples (original + additional) from the video
            for sample in video_data:
                handsign_data.append(sample)

        handsign_data = np.array(handsign_data)
        total_video_samples = handsign_data.shape[0]  # Total samples generated from the videos processed

        # Ensure total number of video samples matches the predefined videos_per_handsign
        if total_video_samples < videos_per_handsign:
            padding_needed = videos_per_handsign - total_video_samples
            print(f"Warning: Padding with {padding_needed} empty samples for handsign '{handsign_folder}'.")
            handsign_data = np.pad(handsign_data, ((0, padding_needed), (0, 0), (0, 0), (0, 0)), mode='constant')
        elif total_video_samples > videos_per_handsign:
            handsign_data = handsign_data[:videos_per_handsign]
            print(f"Warning: Trimming excess samples for handsign '{handsign_folder}'.")

        data.append(handsign_data)

    final_data = np.array(data)
    print(f"Final dataset shape: {final_data.shape}")

    return final_data

    

In [9]:
##          PROCESS VIDEOS DATASET FUNC CALLING         ##
data_array = process_dataset_with_samples(root_path, handsign_names, frames_per_video, num_landmarks, num_coordinates, videos_per_handsign, num_additional_samples)

# Save the data array to a .npy file
np.save('handsigns_data.npy', data_array)
print("Data saved to handsigns_data.npy")
    

Processing videos for handsign 0:   9%|▊         | 13/150 [00:24<03:33,  1.56s/it]

Processing videos for handsign 0:  21%|██▏       | 32/150 [00:46<02:17,  1.17s/it]

Processing videos for handsign 0:  74%|███████▍  | 111/150 [02:13<00:44,  1.14s/it]

Processing videos for handsign 0:  93%|█████████▎| 139/150 [02:44<00:12,  1.12s/it]

Error: frames on NomenclatedDataset - Copy\A\split_43.mp4 are less than 25% of frames_per_video or less than 3. filling with zeros.


...

Processing videos for handsign 36:  45%|████▌     | 68/150 [01:09<01:24,  1.04s/it]

Processing videos for handsign 36:  59%|█████▊    | 88/150 [01:29<01:07,  1.09s/it]

Processing videos for handsign 36:  85%|████████▌ | 128/150 [02:11<00:23,  1.06s/it]

Processing videos for handsign 38:  43%|████▎     | 65/150 [01:08<01:28,  1.04s/it]

Final dataset shape: (39, 300, 8, 65, 3)
Data saved to handsigns_data.npy


In [7]:
##        DATA AUGMENTATION (OPTIONAL)        ##

def apply_rotation(landmarks, angle_degrees_x, angle_degrees_y, angle_degrees_z):
    """Rotate the landmarks in 3D space by given angles around X, Y, and Z axes."""
    angle_radians_x = np.radians(angle_degrees_x)
    angle_radians_y = np.radians(angle_degrees_y)
    angle_radians_z = np.radians(angle_degrees_z)

    # Rotation matrix around the X-axis
    rotation_matrix_x = np.array([
        [1, 0, 0],
        [0, np.cos(angle_radians_x), -np.sin(angle_radians_x)],
        [0, np.sin(angle_radians_x), np.cos(angle_radians_x)]
    ])

    # Rotation matrix around the Y-axis
    rotation_matrix_y = np.array([
        [np.cos(angle_radians_y), 0, np.sin(angle_radians_y)],
        [0, 1, 0],
        [-np.sin(angle_radians_y), 0, np.cos(angle_radians_y)]
    ])

    # Rotation matrix around the Z-axis
    rotation_matrix_z = np.array([
        [np.cos(angle_radians_z), -np.sin(angle_radians_z), 0],
        [np.sin(angle_radians_z), np.cos(angle_radians_z), 0],
        [0, 0, 1]
    ])

    # Combine rotations by multiplying the matrices (Z * Y * X)
    combined_rotation_matrix = np.dot(np.dot(rotation_matrix_z, rotation_matrix_y), rotation_matrix_x)

    return np.dot(landmarks, combined_rotation_matrix)

def apply_scaling(landmarks, scale_factor):
    """Scale the landmarks by a given factor."""
    return landmarks * scale_factor

def apply_translation(landmarks, translation_vector):
    """Translate the landmarks by a given vector (x, y, z)."""
    return landmarks + translation_vector

def add_noise(landmarks, noise_level=0.001):
    """Add random noise to the landmarks."""
    noise = np.random.normal(0, noise_level, landmarks.shape)
    return landmarks + noise

def augment_data(data_array, num_augmented_versions=5):
    """
    Augment the data array by applying transformations.
    Creates `num_augmented_versions` augmented copies of each handsign video.
    """
    
    # Augmentation parameters
    noise_level = 0.005  # for the add_noise() function
    translation_vector = np.random.uniform(-0.05, 0.05, 3)  # for the apply_translation() function
    scale_factor = np.random.uniform(0.6, 1.6)  # for the apply_scaling() function
    angle_degrees_x = np.random.uniform(-20, 20)  # for rotation around X-axis
    angle_degrees_y = np.random.uniform(-20, 20)  # for rotation around Y-axis
    angle_degrees_z = np.random.uniform(-20, 20)  # for rotation around Z-axis

    augmented_data = []
    for handsign_data in data_array:
        augmented_handsign_data = []
        for video_data in handsign_data:
            augmented_videos = [video_data]  # Start with the original video data

            for _ in range(num_augmented_versions):
                augmented_video = []
                for frame in video_data:
                    # Apply a combination of augmentations
                    rotated_frame = apply_rotation(frame, angle_degrees_x, angle_degrees_y, angle_degrees_z)
                    scaled_frame = apply_scaling(rotated_frame, scale_factor)
                    translated_frame = apply_translation(scaled_frame, translation_vector)
                    noisy_frame = add_noise(translated_frame, noise_level)

                    augmented_video.append(noisy_frame)
                
                augmented_videos.append(np.array(augmented_video))

            # Flatten the augmented videos for each original video
            augmented_handsign_data.extend(augmented_videos)

        augmented_data.append(np.array(augmented_handsign_data))
    
    return np.array(augmented_data)

# Load original handsigns data
handsigns_data = np.load('handsigns_data.npy')

# Apply augmentation
if data_augmentation:
    augmented_data = augment_data(handsigns_data, num_augmented_versions)
    # Save the augmented data to a new .npy file
    np.save('handsigns_data_augmented.npy', augmented_data)
    # Update the videos per handsign value to match the videos generated by the augmentation
    data_array = np.load('handsigns_data_augmented.npy')
    videos_per_handsign = data_array.shape[1]
    
    print("Augmented data saved to handsigns_data_augmented.npy, videos_per_handsign augmented by "+str(num_augmented_versions)+" per existing video for a total of "+str(videos_per_handsign)+" videos per handsign")
else:
    print("no data augmentation was performed, check the flag on the first cell")

Augmented data saved to handsigns_data_augmented.npy, videos_per_handsign augmented by 1 per existing video for a total of 600 videos per handsign


In [8]:
##          MODEL DEFINITION            ##
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


# Reshape input to (frames_per_video, num_landmarks * num_coordinates)
new_input_shape = (frames_per_video, num_landmarks * num_coordinates)

model = Sequential([
    # Reshape layer
    Reshape((frames_per_video, num_landmarks * num_coordinates), input_shape=(frames_per_video, num_landmarks, num_coordinates)),
    
    # LSTM layers with Dropout and Batch Normalization to reduce overfitting
    Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))),
    BatchNormalization(),
    Dropout(0.4),
    
    # LSTM layers with Dropout and Batch Normalization to reduce overfitting
    Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))),
    BatchNormalization(),
    Dropout(0.4),
    
    Bidirectional(LSTM(128, return_sequences=False, kernel_regularizer=l2(0.001))),
    BatchNormalization(),
    Dropout(0.3),
    
    # Fully connected layers
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),
    
    # Output layer for multi-class classification
    Dense(num_handsigns, activation='softmax')  # Softmax for multi-class classification
])

# Specify a learning rate
learning_rate = 0.005

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#model.summary() # Uncomment if you want to see the model summary


In [16]:
##        DATA PREPROCESSING FOR TRAINING            ##
from sklearn.model_selection import train_test_split
import shutil

# Load the data from the .npy file, making a copy to use for training as to not modify the original extracted data
if data_augmentation:
    shutil.copy('handsigns_data_augmented.npy', 'handsigns_data_training_copy.npy')
    data_array = np.load('handsigns_data_training_copy.npy')
    
    # Update videos_per_handsign based on augmentation
    #videos_per_handsign = data_array.shape[1]  # Dynamically update based on the new augmented shape
    print("using handsigns_data_augmented.npy. After augmentation videos per handsign updated to: " + str(data_array.shape[1]))
else:
    shutil.copy('handsigns_data.npy', 'handsigns_data_training_copy.npy')
    data_array = np.load('handsigns_data_training_copy.npy')

print('Array used shape: ',data_array.shape)
# X remains unchanged
X = data_array 

# Create labels for each handsign (0 to num_handsigns-1)
# This creates a label for each hand sign, repeated for each video
y = np.repeat(np.arange(num_handsigns), videos_per_handsign)
y = y.reshape(num_handsigns, videos_per_handsign)

# Initialize lists to hold training and validation data
X_train_list = []
X_val_list = []
y_train_list = []
y_val_list = []

# Split videos and labels for each handsign
for handsign_index in range(num_handsigns):
    # Split the videos within each handsign
    train_indices, val_indices = train_test_split(
        np.arange(videos_per_handsign), 
        test_size=0.2, 
        random_state=42
    )
    
    # Select training and validation data for this handsign
    X_train_list.append(data_array[handsign_index, train_indices])
    X_val_list.append(data_array[handsign_index, val_indices])
    
    # Select corresponding labels
    y_train_list.append(y[handsign_index, train_indices])
    y_val_list.append(y[handsign_index, val_indices])

# Concatenate lists to form the final training and validation sets
X_train = np.concatenate(X_train_list, axis=0)
X_val = np.concatenate(X_val_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)
y_val = np.concatenate(y_val_list, axis=0)

# Reshape X_train and X_val to fit the model's expected input shape
X_train = X_train.reshape(-1, frames_per_video, num_landmarks, num_coordinates)
X_val = X_val.reshape(-1, frames_per_video, num_landmarks, num_coordinates)

# Flatten y_train and y_val
y_train = y_train.flatten()
y_val = y_val.flatten()

using handsigns_data_augmented.npy. After augmentation videos per handsign updated to: 600
Array used shape:  (39, 600, 8, 65, 3)


In [14]:
##          MODEL TRAINING          ##
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Enable ITEX optimization
tf.config.optimizer.set_jit(True)

# Callback helpers for model training #
# Early stopping to stop training when validation loss stops improving
# Model checkpointing to save the best model during training
# Reduce learning rate when a metric has stopped improving

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  
checkpoint = ModelCheckpoint('best_handsigns_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)  

# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=1000,
    batch_size=32,
    validation_data=(X_val, y_val), 
    callbacks=[early_stopping, checkpoint, reduce_lr],
    verbose=1
)


# Save the trained model
model.save('handsigns_model.h5')

# Optionally, save the training history
import pickle
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
    

    
##          TRAINING HISTORY ANALYSIS           ##
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

# If you want to save the plot instead of displaying it:
# plt.savefig('training_history.png')

NameError: name 'X_train' is not defined

In [9]:
##          CONTINUOUS PREDICTION WITH SLIDING WINDOW           ##

import collections
import cv2
import mediapipe as mp
import numpy as np 
import tensorflow as tf
from scipy.interpolate import interp1d  # For smoothing landmarks

# Load the trained model
model = tf.keras.models.load_model('best_handsigns_model.keras')

# Initialize Mediapipe solutions outside the loop for efficiency
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, 
                                    max_num_hands=2,
                                    min_detection_confidence=0.4,  # Lowered confidence to allow for fast movement detection
                                    min_tracking_confidence=0.4)   # Lowered tracking confidence
mp_pose = mp.solutions.pose.Pose(static_image_mode=False,
                                 model_complexity=0,
                                 min_detection_confidence=0.4, 
                                 min_tracking_confidence=0.4)

# Open webcam feed
cap = cv2.VideoCapture(0)

# Sliding window buffer for frames
frame_buffer = collections.deque(maxlen=frames_per_video)

# To smooth predictions, keep track of recent predictions
prediction_buffer = collections.deque(maxlen=6)

# Store the last prediction
last_prediction = "No Prediction"
last_confidence = 0.0

# Track missing hands to reset landmarks if missing for too long
hand_missing_threshold = 5
left_hand_missing_count = 0
right_hand_missing_count = 0

# Store the last known hand landmarks to compare in future frames
last_left_hand_landmarks = None
last_right_hand_landmarks = None

landmark_drawing = True

# Smoothing factor for missing landmarks
smoothing_factor = 0.8  # Weight to smooth landmarks during quick movements

# Draw landmarks on the frame
def draw_landmarks(frame, landmarks):
    """Draw landmarks on the frame."""
    for i, (x, y, z) in enumerate(landmarks):
        h, w, _ = frame.shape
        x = int(x * w + 325)
        y = int(y * h + 250)
        
        if i < 21:  # Left hand landmarks
            color = (0, 255, 0)
        elif i < 42:  # Right hand landmarks
            color = (0, 0, 255)
        else:  # Body landmarks
            color = (255, 0, 0)
        
        cv2.circle(frame, (x, y), 5, color, -1)

def smooth_landmarks(new_landmarks, old_landmarks):
    """Smooth landmarks by interpolating between old and new."""
    if old_landmarks is None:
        return new_landmarks

    return old_landmarks * (1 - smoothing_factor) + new_landmarks * smoothing_factor

def internal_process_frame(frame, mp_hands, mp_pose):
    global last_left_hand_landmarks, last_right_hand_landmarks
    global left_hand_missing_count, right_hand_missing_count

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    hands_results = mp_hands.process(image)
    pose_results = mp_pose.process(image)

    # Initialize a zero-filled array for landmarks (51 landmarks, each with x, y, z)
    landmarks = np.zeros((num_landmarks, num_coordinates))

    # Detect nose for relative normalization
    try:
        nose_landmark = pose_results.pose_landmarks.landmark[0]
    except:
        class nose_landmark:
            x = 0
            y = 0
            z = 0
        nose_landmark = nose_landmark()

    if hands_results.multi_hand_landmarks:
        handedness_labels = [hand.classification[0].label for hand in hands_results.multi_handedness]

        for i, hand_landmarks in enumerate(hands_results.multi_hand_landmarks):
            if handedness_labels[i] == 'Left':
                left_hand = np.array([(lm.x - nose_landmark.x, lm.y - nose_landmark.y, lm.z - nose_landmark.z) 
                                      for lm in hand_landmarks.landmark])

                # Smooth the transition between frames
                left_hand = smooth_landmarks(left_hand, last_left_hand_landmarks)
                
                # Update the stored landmarks for the next frame
                last_left_hand_landmarks = left_hand
                landmarks[:21] = left_hand  # Insert the left hand landmarks into the first 21 slots

            elif handedness_labels[i] == 'Right':
                right_hand = np.array([(lm.x - nose_landmark.x, lm.y - nose_landmark.y, lm.z - nose_landmark.z) 
                                       for lm in hand_landmarks.landmark])

                # Smooth the transition between frames
                right_hand = smooth_landmarks(right_hand, last_right_hand_landmarks)
                
                # Update the stored landmarks for the next frame
                last_right_hand_landmarks = right_hand
                landmarks[21:42] = right_hand  # Insert the right hand landmarks into slots 21-42

        # Reset the missing counts if hands are detected
        left_hand_missing_count = 0
        right_hand_missing_count = 0
    else:
        # Increment missing count when hands are not detected
        left_hand_missing_count += 1
        right_hand_missing_count += 1

        # Reuse last known landmarks if available and hands are missing for too long
        if last_left_hand_landmarks is not None:
            landmarks[:21] = last_left_hand_landmarks
        if last_right_hand_landmarks is not None:
            landmarks[21:42] = last_right_hand_landmarks

        # Reset landmarks if hands are missing for too long
        if left_hand_missing_count > hand_missing_threshold:
            last_left_hand_landmarks = None
        if right_hand_missing_count > hand_missing_threshold:
            last_right_hand_landmarks = None

    # Fill in body landmarks (9 selected)
    selected_body_landmarks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
    if pose_results.pose_landmarks:
        for idx, landmark_idx in enumerate(selected_body_landmarks):
            lm = pose_results.pose_landmarks.landmark[landmark_idx]
            landmarks[42 + idx] = (lm.x - nose_landmark.x, lm.y - nose_landmark.y, lm.z - nose_landmark.z)

    return landmarks


# Make a prediction based on the buffer
def predict_handsign(buffer):
    """Make a prediction based on a buffer of frames."""    
    video_data = np.array(buffer)
    video_data = video_data.reshape(1, frames_per_video, num_landmarks, num_coordinates)

    # Make prediction
    prediction = model.predict(video_data, verbose=0)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]

    return predicted_class, confidence
scalar = 1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (int(640*scalar), int(480*scalar)))
    frame = cv2.flip(frame, 1)

    # Process every frame (no skipping)
    landmarks = process_frame(frame, mp_hands, mp_pose)

    # Add the landmarks to the frame buffer
    frame_buffer.append(landmarks)

    # Draw the landmarks on the frame
    if landmark_drawing:
        draw_landmarks(frame, landmarks)

    # Once the buffer is full, make a prediction using the sliding window
    if len(frame_buffer) == frames_per_video:
        predicted_class, confidence = predict_handsign(frame_buffer)
        predicted_handsign = handsign_names.get(predicted_class, f"HandSign_{predicted_class}")

        # Update the last prediction
        last_prediction = predicted_handsign
        last_confidence = confidence

        # Store prediction and confidence in the buffer for smoothing
        #print((predicted_class, confidence))
        prediction_buffer.append((predicted_class, confidence))

        # Average the last N predictions to smooth the output
        avg_pred_class = np.argmax(np.bincount([p[0] for p in prediction_buffer]))
        avg_confidence = np.mean([p[1] for p in prediction_buffer if p[0] == avg_pred_class])

        if avg_confidence > 0.8:
            last_prediction = handsign_names.get(avg_pred_class, f"HandSign_{avg_pred_class}")
            last_confidence = avg_confidence
        elif 0.45 < avg_confidence < 0.8:
            last_prediction = "deteccion insegura: "+handsign_names.get(avg_pred_class, f"HandSign_{avg_pred_class}")
            last_confidence = avg_confidence
        else:
            last_prediction = "deteccion nula"
            last_confidence = avg_confidence
            
    # Display the last prediction on the frame
    cv2.putText(frame, f"Predicted: {last_prediction} Conf: {last_confidence:.2f}", 
                (10, 30), cv2.FONT_ITALIC, 0.7, (0, 0, 0), 2)

    cv2.imshow('Hands Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [14]:
##         V2 CONTINUOUS PREDICTION WITH SLIDING WINDOW           ##
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
from tensorflow.keras.models import load_model


# Parameters for video feed and predictions
show_landmarks = False  # Toggle to show hand landmarks
show_connections = True  # Toggle to show landmark connections
show_prediction_text = True  # Toggle to show prediction and accuracy on screen
accuracy_threshold = 0.80  # Threshold for highlighting low accuracy 
sliding_window_size = frames_per_video  # Size of the sliding window for prediction
smoothing_buffer_size = 4  # Buffer size for prediction smoothing (larger = smoother)
mediapipe_confidence = 0.2  # Confidence threshold for mediapipe detection

# Load the trained model
model = load_model('best_handsigns_model.keras')

# Initialize MediaPipe hands and pose solutions
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands(min_detection_confidence=mediapipe_confidence, min_tracking_confidence=mediapipe_confidence)
pose = mp_pose.Pose(min_detection_confidence=mediapipe_confidence, min_tracking_confidence=mediapipe_confidence)

# Set up sliding window and smoothing buffer
sliding_window = deque(maxlen=sliding_window_size)
smoothing_buffer = deque(maxlen=smoothing_buffer_size)

# Initialize video capture (webcam)
cap = cv2.VideoCapture(0)

# Function to smooth predictions
def smooth_predictions(predictions):
    if len(predictions) > 0:
        return np.mean(predictions, axis=0)
    return np.zeros(num_handsigns)

# Main loop for video feed and prediction
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    #frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    frame = cv2.flip(frame, 1)
    
    # Process frame to extract landmarks and results
    landmarks = process_frame(frame, hands, pose)
    hands_results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    pose_results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Append the landmarks to the sliding window
    sliding_window.append(landmarks)
    
    # Check if the sliding window is full
    if len(sliding_window) == sliding_window_size:
        # Prepare input for the model (reshape and normalize)
        input_data = np.array(sliding_window).reshape(1, sliding_window_size, num_landmarks, num_coordinates)
        
        # Predict the hand sign
        predictions = model.predict(input_data)
        prediction = np.argmax(predictions)
        confidence = np.max(predictions)
        
        # Add the prediction to the smoothing buffer
        smoothing_buffer.append(predictions)
        smoothed_predictions = smooth_predictions(smoothing_buffer)
        smoothed_prediction = np.argmax(smoothed_predictions)
        smoothed_confidence = np.max(smoothed_predictions)

        # Display prediction on screen if enabled
        if show_prediction_text:
            text = f"Sign: {handsign_names[smoothed_prediction]}, Accuracy: {smoothed_confidence:.2f}"
            if smoothed_confidence < accuracy_threshold:
                text = f"Low accuracy ({handsign_names[smoothed_prediction]})"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Draw landmarks if enabled
    if show_landmarks and hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks:
            if show_connections:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            else:
                mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks)
            mp.solutions.drawing_utils.draw_landmarks(frame, pose_results.pose_landmarks)
            

    # Show the frame
    cv2.imshow('Hand Sign Recognition', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
1/1 ━━━━━━━━━━━━

In [18]:
##         TFLITE V2 CONTINUOUS PREDICTION WITH SLIDING WINDOW           ##

import cv2
import mediapipe as mp
import numpy as np
from collections import deque
import tensorflow.lite as tflite  # Or use `from tensorflow.lite import Interpreter` if using full TensorFlow

# Parameters for video feed and predictions
show_landmarks = False
show_connections = True
show_prediction_text = True
accuracy_threshold = 0.80
sliding_window_size = frames_per_video
smoothing_buffer_size = 5
mediapipe_confidence = 0.2

# Load the TFLite model
interpreter = tflite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize MediaPipe hands and pose solutions
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
hands = mp_hands.Hands(min_detection_confidence=mediapipe_confidence, min_tracking_confidence=mediapipe_confidence)
pose = mp_pose.Pose(min_detection_confidence=mediapipe_confidence, min_tracking_confidence=mediapipe_confidence)

# Set up sliding window and smoothing buffer
sliding_window = deque(maxlen=sliding_window_size)
smoothing_buffer = deque(maxlen=smoothing_buffer_size)

# Initialize video capture (webcam)
cap = cv2.VideoCapture(0)

# Function to smooth predictions
def smooth_predictions(predictions):
    if len(predictions) > 0:
        return np.mean(predictions, axis=0)
    return np.zeros(num_handsigns)

# Main loop for video feed and prediction
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    
    # Process frame to extract landmarks and results
    landmarks = process_frame(frame, hands, pose)
    hands_results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    pose_results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Append the landmarks to the sliding window
    sliding_window.append(landmarks)
    
    # Check if the sliding window is full
    if len(sliding_window) == sliding_window_size:
        # Prepare input for the model (reshape and normalize)
        input_data = np.array(sliding_window).reshape(1, sliding_window_size, num_landmarks, num_coordinates).astype(np.float32)
        
        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)
        
        # Run inference
        interpreter.invoke()
        
        # Get predictions
        predictions = interpreter.get_tensor(output_details[0]['index'])
        prediction = np.argmax(predictions)
        confidence = np.max(predictions)
        
        # Add the prediction to the smoothing buffer
        smoothing_buffer.append(predictions)
        smoothed_predictions = smooth_predictions(smoothing_buffer)
        smoothed_prediction = np.argmax(smoothed_predictions)
        smoothed_confidence = np.max(smoothed_predictions)

        # Display prediction on screen if enabled
        if show_prediction_text:
            text = f"Sign: {handsign_names[smoothed_prediction]}, Accuracy: {smoothed_confidence:.2f}"
            if smoothed_confidence < accuracy_threshold:
                text = f"Low accuracy ({handsign_names[smoothed_prediction]})"
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Draw landmarks if enabled
    if show_landmarks and hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks:
            if show_connections:
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            else:
                mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks)
            mp.solutions.drawing_utils.draw_landmarks(frame, pose_results.pose_landmarks)

    # Show the frame
    cv2.imshow('Hand Sign Recognition', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [16]:
##          MODEL METRICS           ##
from sklearn.metrics import classification_report, confusion_matrix, top_k_accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle

# Ensure labels are correctly ordered for the report and confusion matrix
hand_sign_labels = [handsign_names[i] for i in sorted(handsign_names.keys())]

# Modify the analyze_training_history function to use hand_sign_labels
def analyze_training_history(history, model, X_val, y_val, print_graphs=True):
    
    # Print the final training and validation accuracy
    final_train_acc = history['accuracy'][-1]
    final_val_acc = history['val_accuracy'][-1]
    print(f"Final Training Accuracy: {final_train_acc:.2f}")
    print(f"Final Validation Accuracy: {final_val_acc:.2f}")
    
    # Evaluate model performance on validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    
    # Generate predictions and calculate metrics for validation set
    y_pred = np.argmax(model.predict(X_val), axis=1)
    
    # Print Classification Report with handsign names
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred, target_names=hand_sign_labels, digits=4))
    
    # Confusion Matrix
    cm = confusion_matrix(y_val, y_pred)
    print("\nConfusion Matrix:")
    print(cm)
    
    # Compute Top-k Accuracy
    k = 3
    top_k_accuracy = top_k_accuracy_score(y_val, model.predict(X_val), k=k)
    print(f"Top-{k} Accuracy: {top_k_accuracy:.4f}")

    # Per-class ROC-AUC
    y_val_one_hot = tf.keras.utils.to_categorical(y_val, num_classes=len(handsign_names))
    y_pred_proba = model.predict(X_val)
    roc_auc_scores = roc_auc_score(y_val_one_hot, y_pred_proba, multi_class='ovr')
    print(f"Per-Class ROC-AUC Score: {roc_auc_scores:.4f}")
    
    # Cohen's Kappa Score
    y_pred_labels = np.argmax(y_pred_proba, axis=1)
    kappa_score = cohen_kappa_score(y_val, y_pred_labels)
    print(f"Cohen's Kappa Score: {kappa_score:.4f}")

 # Display the Confusion Matrix graphically with better readability if print_graphs is True
    if print_graphs:
        fig, ax = plt.subplots(figsize=(10, 10))  # Increase figure size
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=hand_sign_labels)
        disp.plot(cmap='Blues', ax=ax, xticks_rotation=45)  # Rotate x-axis labels for readability
        plt.title("Confusion Matrix")
        plt.show()
        
    # Plot training history if print_graphs is True
    if print_graphs:
        plt.figure(figsize=(14, 5))
        
        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history['accuracy'], label='Train Accuracy')
        plt.plot(history['val_accuracy'], label='Validation Accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history['loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

# Load the training history and analyze it
def load_training_history(filepath):
    with open(filepath, 'rb') as file:
        history = pickle.load(file)
    return history

history = load_training_history('training_history.pkl')
analyze_training_history(history, model, X_val, y_val, print_graphs=True)


NameError: name 'X_val' is not defined